In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn import svm

from keras.datasets import mnist

# 손글씨 이미지 데이터셋 784 x 1짜리 벡터로 변환 = 28 x 28 pixel로 이루어진 손글씨 이미지
((x_train, y_train), (x_test, y_test)) = mnist.load_data()

X_train = x_train.reshape(-1,28*28)
X_test = x_test.reshape(-1,28*28)

N_train,D = X_train.shape
N_test,D = X_test.shape

# dimensional reduction (PCA) : 시간 단축하려고
# covariance
mu = np.mean(X_train, axis = 0)
C = X_train - mu
C = (1/N_train) * C.T.dot(C)

# SVD: C = U*S*V^T
U, s, V = np.linalg.svd(C, full_matrices = True)
S = np.diag(s)
V = V.T

# 전체 입력 차원 784 중에서, 차원을 축소해서 사용
D_reduce = 30
X_train = X_train.dot(V[:,0:D_reduce]) # N_train x D_reduce
X_test = X_test.dot(V[:,0:D_reduce]) # N_train x D_reduce

X_train = np.hstack([np.ones((N_train,1)),X_train])
X_test = np.hstack([np.ones((N_test,1)),X_test])

# one-of-K coding
num = np.unique(y_train, axis = 0)
num = num.shape[0]

t_train = np.eye(num)[y_train]
t_test = np.eye(num)[y_test]

In [2]:
# parameter
M = np.array([X_train.shape[1],100,1000,10])  # layer별 뉴런 개수. 시작은 input+1(bias), 마지막은 클래스 개수
L = M.shape[0] - 1   # layer 수  

eta = 1e-5
maxEpoch = 30

# initialize parameter
# W[0] = 31 x 100
# W[1] = 100 x 1000
# W[2] = 1000 x 10
W = []

for l in range(L):
    W.append(np.random.randn(M[l],M[l + 1]))
    print(W[l].shape)


(31, 100)
(100, 1000)
(1000, 10)


In [3]:
# activation function
def act(x):        # activation func 
    return np.tanh(x)

def dact(x):       # 도함수
    return 1 - np.tanh(x)**2

# softmax
def softmax(x):    
    
    if x.ndim == 1:
        f_x = np.exp(x)
        return f_x / np.sum(f_x)
    
    elif x.ndim == 2:
        max = np.max(x,axis = 1,keepdims = True)
        e_x = np.exp(x - max)
        sum = np.sum(e_x,axis = 1,keepdims = True)
        f_x = e_x / sum 
        return f_x

# cross entropy
def cross_entropy(y,t):
    N,K = y.shape
    e = np.sum(- t * np.log(y), axis = 1)
    e = np.mean(e)
    
    return e

cost = []
accuracy = []

# initialize for a single input

# forward propagation
# 뉴런 개수만큼
z_sample = []
a_sample = []
for l in range(L):    
    z_sample.append(np.zeros((1,M[l + 1])))
    a_sample.append(np.zeros((1,M[l + 1])))    
pred_sample = softmax(z_sample[-1])   # output y  

# backpropagation
delta = []
for l in np.arange(L):    
    delta.append(np.zeros((M[l],1)))
        
# initialize for entire inputs
# forward propagation
z_train = []
a_train = []
for l in range(L):     # N개 한번에 계산할 준비  
    z_train.append(np.zeros((N_train,M[l + 1])))
    a_train.append(np.zeros((N_train,M[l + 1])))    
pred_train = softmax(z_train[-1])    

In [4]:
print(N_train)
print(X_train.shape)
print(t_train.shape)

60000
(60000, 31)
(60000, 10)


In [14]:
# batch gradient descent
for epoch in range(maxEpoch):  
          
    W_new = W
    
    ### forward propagation
    # input layer
    z_sample[0] = X_train.dot(W[0])    # 60000x31 dot 31x100 = 60000x100
    a_sample[0] = act(z_sample[0])     # 60000x100
    
    # 나머지 layer
    for l in range(L - 1):    
        z_sample[l + 1] = a_sample[l].dot(W[l + 1])   
        a_sample[l + 1] = act(z_sample[l + 1])        

    # output layer
    pred_sample = softmax(z_sample[-1])      

    ### backpropagation : 각 layer마다 delta, parameter 구함
    # output layer
    delta[L - 1] = -(t_train - pred_sample)  # local gradient    

    # propagation
    for l in np.arange(L - 1,1,-1):
        delta[l - 1] = W[l].dot(delta[l].T).T * dact(z_sample[l - 1])         
    
    # input layer        
    delta[0] = W[1].dot(delta[1].T).T * dact(z_sample[0]) 

    ### update
    W_new[L - 1] = W_new[L - 1] - eta*a_sample[L - 2].T.dot(delta[L - 1])
    for l in np.arange(L - 1,1,-1):   
        W_new[l - 1] = W_new[l - 1] - eta*a_sample[l - 2].T.dot(delta[l - 1])  
    # 마지막은 a 대신 x
    W_new[0] = W_new[0] - eta*X_train.T.dot(delta[0])  
    
    W = W_new

    # forward propagation
    z_train[0] = X_train.dot(W[0])
    a_train[0] = act(z_train[0])

    for l in range(L - 1):    
        z_train[l + 1] = a_train[l].dot(W[l + 1])
        a_train[l + 1] = act(z_train[l + 1])

    pred_train = softmax(z_train[-1])

    # performance
    cost.append(cross_entropy(pred_train,t_train))
    accuracy.append(np.sum(y_train == np.argmax(pred_train, axis = 1)) / N_train)
    
    print('[epoch %d] cross entropy: %.4f, accuracy: %.4f'%(epoch,cost[-1],accuracy[-1]))     

[epoch 0] cross entropy: 21.8277, accuracy: 0.3401
[epoch 1] cross entropy: 22.7977, accuracy: 0.3675
[epoch 2] cross entropy: 20.6565, accuracy: 0.4612
[epoch 3] cross entropy: 15.9617, accuracy: 0.4688
[epoch 4] cross entropy: 17.4357, accuracy: 0.4699
[epoch 5] cross entropy: 17.3424, accuracy: 0.4886
[epoch 6] cross entropy: 17.7095, accuracy: 0.5125
[epoch 7] cross entropy: 12.9236, accuracy: 0.5585
[epoch 8] cross entropy: 12.9759, accuracy: 0.5756
[epoch 9] cross entropy: 9.6258, accuracy: 0.6413
[epoch 10] cross entropy: 8.5523, accuracy: 0.6338
[epoch 11] cross entropy: 8.6640, accuracy: 0.6543
[epoch 12] cross entropy: 8.8188, accuracy: 0.6432
[epoch 13] cross entropy: 11.7133, accuracy: 0.5984
[epoch 14] cross entropy: 12.4564, accuracy: 0.6486
[epoch 15] cross entropy: 7.6835, accuracy: 0.6750
[epoch 16] cross entropy: 9.5544, accuracy: 0.6489
[epoch 17] cross entropy: 8.5629, accuracy: 0.6863
[epoch 18] cross entropy: 7.7480, accuracy: 0.6767
[epoch 19] cross entropy: 10.5